# Rottom Tomatos Functions

In [2]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

#from rottentomatoes import rottentomatoes
from rottentomatoes import rottentomatoes

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-01-14 18:14:51.217721


# Get/Parse/Merge/Process Rotten Tomatoes Data

In [3]:
rt = rottentomatoes()

In [4]:
rt.getRottenTomatoesYearlyData(startYear = 2018, endYear = 2018)

  --> This file is 25.7kB.
/Users/tgadfort/Documents/code/movies/rottentomatoes/data/2018.p size -> 26 kB


In [5]:
rt.parseRottenTomatoes(debug=True)

Parsing /Users/tgadfort/Documents/code/movies/rottentomatoes/data/2009.p
Parsing /Users/tgadfort/Documents/code/movies/rottentomatoes/data/1995.p
Parsing /Users/tgadfort/Documents/code/movies/rottentomatoes/data/2003.p
Parsing /Users/tgadfort/Documents/code/movies/rottentomatoes/data/romance.p
Parsing /Users/tgadfort/Documents/code/movies/rottentomatoes/data/documentary.p
Parsing /Users/tgadfort/Documents/code/movies/rottentomatoes/data/kids__family.p
Parsing /Users/tgadfort/Documents/code/movies/rottentomatoes/data/2007.p
Parsing /Users/tgadfort/Documents/code/movies/rottentomatoes/data/1991.p
Parsing /Users/tgadfort/Documents/code/movies/rottentomatoes/data/2017.p
Parsing /Users/tgadfort/Documents/code/movies/rottentomatoes/data/1981.p
Parsing /Users/tgadfort/Documents/code/movies/rottentomatoes/data/1985.p
Parsing /Users/tgadfort/Documents/code/movies/rottentomatoes/data/2013.p
Parsing /Users/tgadfort/Documents/code/movies/rottentomatoes/data/1996.p
Parsing /Users/tgadfort/Documents